In [ ]:
# Install dependencies
!pip install torch flask pyngrok sentence-transformers python-docx pdfplumber nltk --quiet

# Imports
from flask import Flask, render_template_string, request, jsonify
from pyngrok import ngrok
from sentence_transformers import SentenceTransformer, util
from docx import Document
import pdfplumber, os, tempfile, re, nltk
import time
import torch

nltk.download('punkt_tab', quiet=True)

app = Flask(__name__)
model = SentenceTransformer('all-mpnet-base-v2')

# Ngrok for public access - USING YOUR AUTH TOKEN
!ngrok config add-authtoken 34SCZ57JYth1j46CDpD4E8W43zA_4jkDsoPZrENPNBZHmPdV5
public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)

# Helper function to read file
def read_file(file):
    if not file or not file.filename:
        return ""

    ext = os.path.splitext(file.filename)[1].lower()
    with tempfile.NamedTemporaryFile(delete=False, suffix=ext) as tmp:
        tmp.write(file.read())
        path = tmp.name
    text=""
    try:
        if ext==".txt":
            with open(path,"r",encoding="utf-8") as f: text=f.read()
        elif ext==".pdf":
            with pdfplumber.open(path) as pdf:
                for page in pdf.pages: text += page.extract_text() or ""
        elif ext==".docx":
            doc = Document(path)
            text = "\n".join([p.text for p in doc.paragraphs])
    except Exception as e:
        print(f"Error reading file {file.filename}: {e}")
        return ""
    finally:
        os.remove(path)
    return text.strip()

# Function to highlight similar text segments
def highlight_similar_text(source_text, compare_text, similarity_threshold=0.7):
    """
    Highlight similar text segments between source and compare text
    """
    try:
        # Split texts into sentences
        source_sentences = nltk.sent_tokenize(source_text)
        compare_sentences = nltk.sent_tokenize(compare_text)

        if not source_sentences or not compare_sentences:
            return source_text, compare_text, []

        # Encode sentences
        source_embs = model.encode(source_sentences, convert_to_tensor=True)
        compare_embs = model.encode(compare_sentences, convert_to_tensor=True)

        # Find similar sentences
        similar_segments = []
        highlighted_source = source_text
        highlighted_compare = compare_text

        for i, source_sent in enumerate(source_sentences):
            similarities = util.cos_sim(source_embs[i], compare_embs)
            max_similarity, max_idx = torch.max(similarities, dim=1)
            max_similarity = max_similarity.item()
            max_idx = max_idx.item()

            if max_similarity >= similarity_threshold:
                compare_sent = compare_sentences[max_idx]

                # Add to similar segments
                similar_segments.append({
                    'source_sentence': source_sent,
                    'compare_sentence': compare_sent,
                    'similarity': max_similarity
                })

                # Highlight in source text
                highlighted_source = highlighted_source.replace(
                    source_sent,
                    f'<span class="plagiarism-highlight">{source_sent}</span>'
                )

                # Highlight in compare text
                highlighted_compare = highlighted_compare.replace(
                    compare_sent,
                    f'<span class="plagiarism-highlight">{compare_sent}</span>'
                )

        return highlighted_source, highlighted_compare, similar_segments

    except Exception as e:
        print(f"Error in highlighting: {e}")
        return source_text, compare_text, []

# HTML Template with Plagiarism Highlighting
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Academic Integrity Predictor</title>
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css" rel="stylesheet">
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Playfair+Display:wght@400;700&family=Inter:wght@300;400;600;700&display=swap');

        :root {
            --gold: #FFD700;
            --dark-gold: #B8860B;
            --black: #0A0A0A;
            --dark-gray: #1A1A1A;
            --medium-gray: #2A2A2A;
            --light-gray: #3A3A3A;
            --neon-gold: #FFEE58;
            --plagiarism-red: #ff4444;
            --plagiarism-orange: #ff8800;
        }

        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Inter', sans-serif;
            background: linear-gradient(135deg, var(--black) 0%, #1a1a1a 100%);
            min-height: 100vh;
            padding: 20px;
            color: white;
            overflow-x: hidden;
        }

        .container {
            max-width: 1400px;
            margin: 0 auto;
            background: rgba(26, 26, 26, 0.95);
            border-radius: 20px;
            box-shadow: 0 25px 50px rgba(0,0,0,0.5),
                        0 0 100px rgba(255, 215, 0, 0.1);
            overflow: hidden;
            border: 1px solid rgba(255, 215, 0, 0.2);
        }

        .header {
            background: linear-gradient(135deg, var(--black) 0%, var(--dark-gray) 100%);
            color: var(--gold);
            padding: 40px 30px;
            text-align: center;
            border-bottom: 2px solid var(--gold);
        }

        .header h1 {
            font-family: 'Playfair Display', serif;
            font-size: 3em;
            margin-bottom: 15px;
            background: linear-gradient(45deg, var(--gold), var(--neon-gold));
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
        }

        .content {
            padding: 40px;
        }

        .mode-tabs {
            display: flex;
            gap: 10px;
            margin-bottom: 40px;
            background: var(--dark-gray);
            padding: 10px;
            border-radius: 15px;
        }

        .mode-tab {
            flex: 1;
            padding: 15px 20px;
            border: 2px solid var(--medium-gray);
            background: transparent;
            color: var(--gold);
            border-radius: 10px;
            font-size: 16px;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.3s ease;
            text-align: center;
        }

        .mode-tab.active {
            background: linear-gradient(135deg, var(--gold), var(--dark-gold));
            color: var(--black);
            border-color: var(--gold);
        }

        .tab-content {
            display: none;
        }

        .tab-content.active {
            display: block;
        }

        .input-section {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 40px;
            margin-bottom: 40px;
        }

        .text-input {
            border: 2px solid var(--medium-gray);
            border-radius: 15px;
            padding: 25px;
            background: rgba(42, 42, 42, 0.8);
        }

        .text-input h3 {
            margin-bottom: 20px;
            color: var(--gold);
            font-family: 'Playfair Display', serif;
            font-size: 1.5em;
        }

        textarea {
            width: 100%;
            height: 250px;
            padding: 20px;
            border: 1px solid var(--light-gray);
            border-radius: 10px;
            resize: vertical;
            font-family: 'Inter', sans-serif;
            font-size: 14px;
            background: var(--dark-gray);
            color: white;
        }

        .source-compare-section {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 30px;
            margin-bottom: 40px;
        }

        .upload-panel {
            border: 2px solid var(--medium-gray);
            border-radius: 15px;
            padding: 25px;
            background: rgba(42, 42, 42, 0.8);
            min-height: 400px;
        }

        .upload-panel.source-panel {
            border-color: var(--neon-gold);
            background: rgba(255, 238, 88, 0.05);
        }

        .panel-header {
            text-align: center;
            margin-bottom: 20px;
        }

        .panel-header h3 {
            color: var(--gold);
            font-family: 'Playfair Display', serif;
            font-size: 1.4em;
            margin-bottom: 10px;
        }

        .upload-area {
            border: 3px dashed var(--light-gray);
            border-radius: 12px;
            padding: 30px 20px;
            text-align: center;
            background: rgba(58, 58, 58, 0.3);
            cursor: pointer;
            margin-bottom: 20px;
        }

        .file-list {
            margin-top: 20px;
            max-height: 200px;
            overflow-y: auto;
        }

        .file-item {
            background: var(--dark-gray);
            padding: 12px 15px;
            margin: 8px 0;
            border-radius: 8px;
            border-left: 4px solid var(--gold);
            display: flex;
            justify-content: space-between;
            align-items: center;
        }

        .remove-file {
            background: #dc3545;
            color: white;
            border: none;
            border-radius: 50%;
            width: 25px;
            height: 25px;
            cursor: pointer;
        }

        .analyze-btn {
            width: 100%;
            padding: 20px;
            background: linear-gradient(135deg, var(--gold), var(--dark-gold));
            color: var(--black);
            border: none;
            border-radius: 15px;
            font-size: 1.3em;
            font-weight: bold;
            cursor: pointer;
            margin-top: 20px;
        }

        .analyze-btn:disabled {
            background: var(--medium-gray);
            color: var(--light-gray);
            cursor: not-allowed;
        }

        .results-section {
            margin-top: 40px;
            padding: 30px;
            background: rgba(42, 42, 42, 0.8);
            border-radius: 20px;
            border: 2px solid var(--gold);
        }

        .similarity-score {
            text-align: center;
            margin-bottom: 40px;
        }

        .score-circle {
            width: 180px;
            height: 180px;
            border-radius: 50%;
            margin: 0 auto 20px;
            display: flex;
            align-items: center;
            justify-content: center;
            position: relative;
            background: conic-gradient(
                from 0deg,
                #ff4444 0%,
                #ffaa00 25%,
                #ffff00 50%,
                #aaff00 75%,
                #00ff00 100%
            );
        }

        .score-circle::before {
            content: '';
            position: absolute;
            width: 160px;
            height: 160px;
            background: var(--dark-gray);
            border-radius: 50%;
        }

        .score-inner {
            width: 140px;
            height: 140px;
            background: var(--black);
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 2em;
            font-weight: bold;
            color: var(--gold);
            z-index: 1;
            border: 3px solid var(--gold);
        }

        .comparison-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(350px, 1fr));
            gap: 20px;
            margin-top: 20px;
        }

        .comparison-card {
            background: var(--dark-gray);
            padding: 20px;
            border-radius: 15px;
            border: 1px solid var(--medium-gray);
            cursor: pointer;
        }

        .similarity-badge {
            font-size: 1.2em;
            font-weight: bold;
            padding: 10px 20px;
            border-radius: 20px;
            text-align: center;
            margin-bottom: 15px;
        }

        .badge-high {
            background: #dc3545;
            color: white;
        }

        .badge-medium {
            background: #ffc107;
            color: black;
        }

        .badge-low {
            background: #28a745;
            color: white;
        }

        /* PLAGIARISM HIGHLIGHTING STYLES */
        .plagiarism-highlight {
            background: linear-gradient(135deg, var(--plagiarism-red), var(--plagiarism-orange));
            color: white;
            padding: 2px 4px;
            border-radius: 3px;
            font-weight: bold;
            animation: pulse-highlight 2s infinite;
        }

        @keyframes pulse-highlight {
            0%, 100% { opacity: 1; }
            50% { opacity: 0.8; }
        }

        .highlighted-text-container {
            background: var(--dark-gray);
            padding: 20px;
            border-radius: 10px;
            border: 1px solid var(--medium-gray);
            margin-top: 15px;
        }

        .text-comparison {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 20px;
        }

        .text-column {
            background: var(--medium-gray);
            padding: 15px;
            border-radius: 8px;
            border: 1px solid var(--light-gray);
        }

        .text-column h6 {
            color: var(--neon-gold);
            margin-bottom: 10px;
            border-bottom: 1px solid var(--light-gray);
            padding-bottom: 5px;
        }

        .highlighted-text {
            max-height: 300px;
            overflow-y: auto;
            line-height: 1.6;
            font-size: 14px;
        }

        .expand-btn {
            background: var(--gold);
            color: var(--black);
            border: none;
            border-radius: 5px;
            padding: 8px 15px;
            cursor: pointer;
            margin-top: 10px;
            font-weight: bold;
        }

        .comparison-header {
            text-align: center;
            margin-bottom: 20px;
            padding: 15px;
            background: rgba(255,215,0,0.1);
            border-radius: 10px;
        }

        .hidden {
            display: none;
        }

        .progress {
            height: 25px;
            margin-bottom: 20px;
            background: var(--medium-gray);
            border-radius: 10px;
            overflow: hidden;
        }

        .progress-bar {
            height: 100%;
            background: linear-gradient(90deg, var(--gold), var(--neon-gold));
            display: flex;
            align-items: center;
            justify-content: center;
            font-weight: bold;
            color: var(--black);
        }

        @media (max-width: 768px) {
            .source-compare-section,
            .input-section,
            .text-comparison {
                grid-template-columns: 1fr;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1><i class="fas fa-graduation-cap"></i> Academic Integrity Predictor</h1>
            <p>Advanced plagiarism detection with text highlighting</p>
        </div>

        <div class="content">
            <div class="mode-tabs">
                <button class="mode-tab active" onclick="switchTab('text')">Text Input</button>
                <button class="mode-tab" onclick="switchTab('one-to-one')">One-to-One Files</button>
                <button class="mode-tab" onclick="switchTab('one-to-many')">One-to-Many Files</button>
            </div>

            <!-- Text Input Tab -->
            <div id="text-tab" class="tab-content active">
                <div class="input-section">
                    <div class="text-input">
                        <h3>Original Text</h3>
                        <textarea id="text1" placeholder="Enter original text..."></textarea>
                    </div>
                    <div class="text-input">
                        <h3>Text to Check</h3>
                        <textarea id="text2" placeholder="Enter text to check..."></textarea>
                    </div>
                </div>
                <button class="analyze-btn" onclick="analyzeText()">Analyze Text Similarity</button>
            </div>

            <!-- One-to-One File Upload Tab -->
            <div id="one-to-one-tab" class="tab-content">
                <div class="input-section">
                    <div class="text-input">
                        <h3>Document 1</h3>
                        <input type="file" id="file1" accept=".txt,.pdf,.docx" style="width: 100%; padding: 10px;">
                    </div>
                    <div class="text-input">
                        <h3>Document 2</h3>
                        <input type="file" id="file2" accept=".txt,.pdf,.docx" style="width: 100%; padding: 10px;">
                    </div>
                </div>
                <button class="analyze-btn" onclick="analyzeOneToOne()">Compare Two Documents</button>
            </div>

            <!-- One-to-Many File Upload Tab -->
            <div id="one-to-many-tab" class="tab-content">
                <div class="source-compare-section">
                    <div class="upload-panel source-panel">
                        <div class="panel-header">
                            <h3>Source Document</h3>
                            <p>Upload ONE reference file</p>
                        </div>
                        <input type="file" id="sourceFileInput" accept=".txt,.pdf,.docx" style="width: 100%; padding: 10px; margin-bottom: 15px;">
                        <div class="file-list" id="sourceFileList">
                            <p class="text-center">No source file selected</p>
                        </div>
                    </div>
                    <div class="upload-panel compare-panel">
                        <div class="panel-header">
                            <h3>Documents to Compare</h3>
                            <p>Upload MULTIPLE files to check</p>
                        </div>
                        <input type="file" id="compareFileInput" multiple accept=".txt,.pdf,.docx" style="width: 100%; padding: 10px; margin-bottom: 15px;">
                        <div class="file-list" id="compareFileList">
                            <p class="text-center">No files to compare</p>
                        </div>
                    </div>
                </div>
                <button class="analyze-btn" id="analyzeManyBtn" onclick="analyzeOneToMany()" disabled>Compare Source vs All Documents</button>
            </div>

            <div class="progress">
                <div class="progress-bar" id="progressBar" style="width: 0%">0%</div>
            </div>

            <div id="results" class="results-section hidden">
                <div id="textResults">
                    <div class="similarity-score">
                        <div class="score-circle">
                            <div class="score-inner">
                                <span id="scorePercent">0%</span>
                            </div>
                        </div>
                        <div style="color: var(--gold); font-size: 1.4em; font-weight: bold;">Similarity Score</div>
                    </div>
                    <div id="textHighlighting"></div>
                </div>
                <div id="oneToOneResults" style="display: none;">
                    <div class="similarity-score">
                        <div class="score-circle">
                            <div class="score-inner">
                                <span id="fileScorePercent">0%</span>
                            </div>
                        </div>
                        <div style="color: var(--gold); font-size: 1.4em; font-weight: bold;">Similarity Score</div>
                    </div>
                    <div id="oneToOneHighlighting"></div>
                </div>
                <div id="oneToManyResults" style="display: none;">
                    <div class="comparison-header">
                        <h4>Comparison Results</h4>
                        <p>Source Document vs All Comparison Files</p>
                    </div>
                    <div class="comparison-grid" id="comparisonGrid"></div>
                </div>
            </div>
        </div>
    </div>

    <script>
        let uploadedFiles = {
            source: null,
            compare: []
        };

        function switchTab(tab) {
            document.querySelectorAll('.mode-tab').forEach(t => t.classList.remove('active'));
            event.target.classList.add('active');
            document.querySelectorAll('.tab-content').forEach(c => c.classList.remove('active'));
            document.getElementById(tab + '-tab').classList.add('active');
            document.getElementById('results').classList.add('hidden');
        }

        // File handling for one-to-many
        document.getElementById('sourceFileInput').addEventListener('change', function(e) {
            if (e.target.files.length > 0) {
                uploadedFiles.source = e.target.files[0];
                updateSourceFileList();
            }
        });

        document.getElementById('compareFileInput').addEventListener('change', function(e) {
            uploadedFiles.compare = Array.from(e.target.files);
            updateCompareFileList();
        });

        function updateSourceFileList() {
            const fileList = document.getElementById('sourceFileList');
            if (uploadedFiles.source) {
                fileList.innerHTML = `
                    <div class="file-item">
                        <span>${uploadedFiles.source.name}</span>
                        <button class="remove-file" onclick="removeSourceFile()">×</button>
                    </div>
                `;
            } else {
                fileList.innerHTML = '<p class="text-center">No source file selected</p>';
            }
            updateAnalyzeButton();
        }

        function updateCompareFileList() {
            const fileList = document.getElementById('compareFileList');
            if (uploadedFiles.compare.length > 0) {
                fileList.innerHTML = uploadedFiles.compare.map((file, index) => `
                    <div class="file-item">
                        <span>${file.name}</span>
                        <button class="remove-file" onclick="removeCompareFile(${index})">×</button>
                    </div>
                `).join('');
            } else {
                fileList.innerHTML = '<p class="text-center">No files to compare</p>';
            }
            updateAnalyzeButton();
        }

        function removeSourceFile() {
            uploadedFiles.source = null;
            updateSourceFileList();
        }

        function removeCompareFile(index) {
            uploadedFiles.compare.splice(index, 1);
            updateCompareFileList();
        }

        function updateAnalyzeButton() {
            document.getElementById('analyzeManyBtn').disabled = !(uploadedFiles.source && uploadedFiles.compare.length > 0);
        }

        // Analysis functions
        async function analyzeText() {
            const text1 = document.getElementById('text1').value;
            const text2 = document.getElementById('text2').value;
            if (!text1 || !text2) {
                alert('Please enter both texts.');
                return;
            }

            const progressBar = document.getElementById('progressBar');
            progressBar.style.width = '50%';
            progressBar.textContent = '50%';

            const formData = new FormData();
            formData.append('text1', text1);
            formData.append('text2', text2);

            try {
                const response = await fetch('/analyze-text', {method: 'POST', body: formData});
                const data = await response.json();
                progressBar.style.width = '100%';
                progressBar.textContent = '100%';
                displayTextResults(data);
            } catch (error) {
                alert('Error analyzing text.');
            }
        }

        async function analyzeOneToOne() {
            const file1 = document.getElementById('file1').files[0];
            const file2 = document.getElementById('file2').files[0];
            if (!file1 || !file2) {
                alert('Please select two files.');
                return;
            }

            const progressBar = document.getElementById('progressBar');
            progressBar.style.width = '50%';

            const formData = new FormData();
            formData.append('file1', file1);
            formData.append('file2', file2);

            try {
                const response = await fetch('/analyze-one-to-one', {method: 'POST', body: formData});
                const data = await response.json();
                progressBar.style.width = '100%';
                displayOneToOneResults(data);
            } catch (error) {
                alert('Error analyzing files.');
            }
        }

        async function analyzeOneToMany() {
            if (!uploadedFiles.source || uploadedFiles.compare.length === 0) {
                alert('Please select source and comparison files.');
                return;
            }

            const progressBar = document.getElementById('progressBar');
            progressBar.style.width = '50%';

            const formData = new FormData();
            formData.append('source_file', uploadedFiles.source);
            uploadedFiles.compare.forEach(file => formData.append('compare_files', file));

            try {
                const response = await fetch('/analyze-one-to-many', {method: 'POST', body: formData});
                const data = await response.json();
                progressBar.style.width = '100%';
                displayOneToManyResults(data.results);
            } catch (error) {
                alert('Error analyzing documents.');
            }
        }

        function displayTextResults(data) {
            document.getElementById('scorePercent').textContent = data.percent.toFixed(1) + '%';
            const highlighting = document.getElementById('textHighlighting');

            if (data.highlighted_source && data.highlighted_compare) {
                highlighting.innerHTML = `
                    <div class="highlighted-text-container">
                        <div class="text-comparison">
                            <div class="text-column">
                                <h6>Original Text (Highlighted):</h6>
                                <div class="highlighted-text">${data.highlighted_source}</div>
                            </div>
                            <div class="text-column">
                                <h6>Compared Text (Highlighted):</h6>
                                <div class="highlighted-text">${data.highlighted_compare}</div>
                            </div>
                        </div>
                        <div style="text-align: center; margin-top: 15px;">
                            <small style="color: #ccc;">
                                <span class="plagiarism-highlight" style="display: inline-block; width: 15px; height: 15px; margin-right: 5px;"></span>
                                Highlighted text shows plagiarized/similar content
                            </small>
                        </div>
                    </div>
                `;
            }

            document.getElementById('textResults').style.display = 'block';
            document.getElementById('oneToOneResults').style.display = 'none';
            document.getElementById('oneToManyResults').style.display = 'none';
            document.getElementById('results').classList.remove('hidden');
        }

        function displayOneToOneResults(data) {
            document.getElementById('fileScorePercent').textContent = data.percent.toFixed(1) + '%';
            const highlighting = document.getElementById('oneToOneHighlighting');

            if (data.highlighted_source && data.highlighted_compare) {
                highlighting.innerHTML = `
                    <div class="highlighted-text-container">
                        <div class="text-comparison">
                            <div class="text-column">
                                <h6>${data.file1} (Highlighted):</h6>
                                <div class="highlighted-text">${data.highlighted_source}</div>
                            </div>
                            <div class="text-column">
                                <h6>${data.file2} (Highlighted):</h6>
                                <div class="highlighted-text">${data.highlighted_compare}</div>
                            </div>
                        </div>
                    </div>
                `;
            }

            document.getElementById('textResults').style.display = 'none';
            document.getElementById('oneToOneResults').style.display = 'block';
            document.getElementById('oneToManyResults').style.display = 'none';
            document.getElementById('results').classList.remove('hidden');
        }

        function displayOneToManyResults(results) {
            const grid = document.getElementById('comparisonGrid');
            grid.innerHTML = results.map((result, index) => {
                const badgeClass = result.similarity >= 70 ? 'badge-high' : result.similarity >= 30 ? 'badge-medium' : 'badge-low';
                return `
                    <div class="comparison-card">
                        <div class="similarity-badge ${badgeClass}">
                            ${result.similarity.toFixed(1)}% Similarity
                        </div>
                        <div style="margin-bottom: 10px;">
                            <strong>Source:</strong> ${result.source}<br>
                            <strong>Compared With:</strong> ${result.compared_with}
                        </div>
                        ${result.highlighted_source && result.highlighted_compare ? `
                            <button class="expand-btn" onclick="toggleHighlighting(${index})">
                                Show Plagiarized Text
                            </button>
                            <div id="highlighting-${index}" class="hidden">
                                <div class="text-comparison">
                                    <div class="text-column">
                                        <h6>Source Text:</h6>
                                        <div class="highlighted-text">${result.highlighted_source}</div>
                                    </div>
                                    <div class="text-column">
                                        <h6>Compared Text:</h6>
                                        <div class="highlighted-text">${result.highlighted_compare}</div>
                                    </div>
                                </div>
                            </div>
                        ` : ''}
                    </div>
                `;
            }).join('');

            document.getElementById('textResults').style.display = 'none';
            document.getElementById('oneToOneResults').style.display = 'none';
            document.getElementById('oneToManyResults').style.display = 'block';
            document.getElementById('results').classList.remove('hidden');
        }

        function toggleHighlighting(index) {
            const div = document.getElementById(`highlighting-${index}`);
            const btn = event.target;
            if (div.classList.contains('hidden')) {
                div.classList.remove('hidden');
                btn.textContent = 'Hide Plagiarized Text';
            } else {
                div.classList.add('hidden');
                btn.textContent = 'Show Plagiarized Text';
            }
        }

        // Initialize
        updateSourceFileList();
        updateCompareFileList();
    </script>
</body>
</html>
'''

@app.route("/", methods=["GET"])
def index():
    return render_template_string(html_template)

@app.route("/analyze-text", methods=["POST"])
def analyze_text():
    try:
        text1 = request.form.get("text1", "")
        text2 = request.form.get("text2", "")

        if not text1 or not text2:
            return jsonify({"error": "Please provide both texts"}), 400

        # Calculate similarity
        emb1 = model.encode(text1, convert_to_tensor=True)
        emb2 = model.encode(text2, convert_to_tensor=True)
        score = util.cos_sim(emb1, emb2).item()
        percent = score * 100

        # Highlight similar text
        highlighted_source, highlighted_compare, similar_segments = highlight_similar_text(text1, text2)

        return jsonify({
            "score": score,
            "percent": percent,
            "highlighted_source": highlighted_source,
            "highlighted_compare": highlighted_compare
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/analyze-one-to-one", methods=["POST"])
def analyze_one_to_one():
    try:
        file1 = request.files.get("file1")
        file2 = request.files.get("file2")

        if not file1 or not file2:
            return jsonify({"error": "Please upload both files"}), 400

        # Read files
        text1 = read_file(file1)
        text2 = read_file(file2)

        if not text1 or not text2:
            return jsonify({"error": "Could not read file contents"}), 400

        # Calculate similarity
        emb1 = model.encode(text1, convert_to_tensor=True)
        emb2 = model.encode(text2, convert_to_tensor=True)
        score = util.cos_sim(emb1, emb2).item()
        percent = score * 100

        # Highlight similar text
        highlighted_source, highlighted_compare, similar_segments = highlight_similar_text(text1, text2)

        return jsonify({
            "score": score,
            "percent": percent,
            "file1": file1.filename,
            "file2": file2.filename,
            "highlighted_source": highlighted_source,
            "highlighted_compare": highlighted_compare
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/analyze-one-to-many", methods=["POST"])
def analyze_one_to_many():
    try:
        source_file = request.files.get("source_file")
        compare_files = request.files.getlist("compare_files")

        if not source_file:
            return jsonify({"error": "Please upload a source file"}), 400

        if len(compare_files) == 0:
            return jsonify({"error": "Please upload at least one file to compare against"}), 400

        # Read source file
        source_content = read_file(source_file)
        if not source_content:
            return jsonify({"error": "Could not read source file content"}), 400

        # Read compare files and calculate similarities
        results = []
        for file in compare_files:
            if file and file.filename:
                content = read_file(file)
                if content:
                    # Calculate similarity
                    source_emb = model.encode(source_content, convert_to_tensor=True)
                    compare_emb = model.encode(content, convert_to_tensor=True)
                    score = util.cos_sim(source_emb, compare_emb).item()
                    similarity = score * 100

                    # Highlight similar text
                    highlighted_source, highlighted_compare, similar_segments = highlight_similar_text(
                        source_content, content
                    )

                    results.append({
                        'source': source_file.filename,
                        'compared_with': file.filename,
                        'similarity': similarity,
                        'highlighted_source': highlighted_source,
                        'highlighted_compare': highlighted_compare
                    })

        return jsonify({"results": results})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(port=5000, use_reloader=False)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌍 Public URL: NgrokTunnel: "https://e0689ba56e0c.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 08:46:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 08:46:46] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 08:46:57] "POST /analyze-text HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 08:56:07] "POST /analyze-text HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 09:01:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 09:03:54] "POST /analyze-one-to-many HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 09:06:17] "POST /analyze-one-to-many HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 09:06:47] "POST /analyze-one-to-many HTTP/1.1" 200 -
